# Face recognition with ORL

In [275]:
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image
import os
import dlib
import cv2
import random

In [276]:
dsFolderPath = "att_faces" # папка датасета
rows = 0 # Количество строк матрицы имен файлов(количество фотографий)
numberPhotoOnePerson = 0
digitalSurname = ""
photoname = ""
mmds = ""

###  Работаем с датасетом

In [277]:
def pathesOfImagesFromDs(dsFolderPath):
    global rows
    global numberPhotoOnePerson
    listOfSubfolders = os.listdir(dsFolderPath)
    photos = list()
    for i in listOfSubfolders:
        listOfImagesInCurrentSubfolder = os.listdir(dsFolderPath + "/" + i)
        for j in listOfImagesInCurrentSubfolder:            
            photos.append(dsFolderPath + "/" + i + "/" + j)
    rows = len(photos)
    numberPhotoOnePerson = len (listOfSubfolders)
    return rows, numberPhotoOnePerson, photos

In [278]:
#pathesOfImagesFromDs(dsFolderPath)

### Создание цифровых фамилий

In [279]:
def initDigitalSurname():
    global digitalSurname
    global number_ds
    number_ds = numberPhotoOnePerson
    numberDigitalSurname = rows / number_ds # под одной цифровой фамилией будет
    digitalSurname = [0] * rows
    for i in range (0, number_ds, 1):
        for j in range (0, int(numberDigitalSurname), 1):
            digitalSurname [int(str(i) + str(j))] = i + 1
    return number_ds, numberDigitalSurname, len(digitalSurname), digitalSurname

In [280]:
#initDigitalSurname()

### Функция получает данные фото (размер)

In [281]:
def getSizePhoto ():
    img = np.array(pathesOfImagesFromDs(dsFolderPath)[2])
    i = 0 # самое первое фото
    img2 = cv2.imread(img[i]) 
    h = img2.shape[0]
    w = img2.shape[1]
    c = img2.shape[2]
    n = h * w
    m = rows    
    return m, n

In [282]:
#getSizePhoto ()

### Распарсиваем датасет в MMDS. Сначала метод. Потом вызываем его "m" раз

In [283]:
# распарсивает одно фото в строку
def parsePhoto (i):
    im=np.array(Image.open(pathesOfImagesFromDs(dsFolderPath)[2][i]))
    im=im.reshape(1,im.shape[0]*im.shape[1])    
    return im

In [284]:
# вызывает по очереди строки в mmds
def create_MMDS():
    m = getSizePhoto ()[0]
    n = getSizePhoto ()[1]
    global mmds
    mmds = np.zeros((m,n)) 
    for i in range (0, m):
        mmds [i] = (parsePhoto (i))/255 #сюды парсить фото построчно и делим их на 255
    return mmds

In [285]:
#create_MMDS()

###  Normalize
    1 - sample
    2 - feature


In [286]:
def nStandDev(mmds):
    mode = 1
    if (mode==1): # sample
        for i in range(0, len(mmds)):
            mmds[i] = (mmds[i])/np.std(mmds[i]) #
    elif(mode==2):   # feature
        for i in range(0, len(mmds[0])):
            mmds[:,i] = (mmds[:,i])/np.std(mmds[:,i]) #
    else:
        mmds = mmds
    return mmds

In [287]:
#nStandDev(create_MMDS())

### Split mmds

In [288]:
def numberSplitMMDS (numberTrain):
    numberTest = 10 - numberTrain
    return numberTrain, numberTest

In [289]:
#nT = input ("Сколько фото из каждой персоны взять для тренировки? ")
#numberSplitMMDS (8)

In [290]:
def splitMMDS ():
    n = getSizePhoto ()[1] 
    m = getSizePhoto ()[0]
    i_trainMMDS = 0
    i_testMMDS = 0

    tM = 9
    mmds = nStandDev(create_MMDS())
    testM = int(initDigitalSurname()[1]) - tM
    trainMMDS = np.zeros(((tM * number_ds), n))
    vyb = int(int(initDigitalSurname()[1]) / testM)
    trainDigitalSurname = list()
    testMMDS = np.zeros(((testM * number_ds), n))
    testDigitalSurname = list()
    
    for i in range(0, m):
        if (i%vyb != 0):
            trainMMDS[i_trainMMDS,:] = mmds[i,:]
            i_trainMMDS += 1       
            trainDigitalSurname.append(digitalSurname[i])
        else:
            testMMDS[i_testMMDS,:] = mmds[i,:]    
            i_testMMDS += 1    
            testDigitalSurname.append(digitalSurname[i])
           
    return trainMMDS, trainDigitalSurname, testMMDS, testDigitalSurname

In [291]:
#splitMMDS ()

In [292]:
def evklidDistanse (vector1, vector2):
    return np.linalg.norm(vector2 - vector1)

In [293]:
def minDist():
    mmds1 = splitMMDS()[0] # База эталонныйх фото
    trainDS = splitMMDS()[1] # База фамилий эталонных фото
    mmds2 = splitMMDS()[2] # База испытуемых фото
    testDS = splitMMDS()[3] # База фамилий испытуемых фото
    acc = 0 # Количество правильно распознаных фото
    vector1 = np.zeros (len(mmds1)) # массив строк из теста
    vector2 = np.zeros (len(mmds2)) # массив строк из трейна
    iTest = 0
    for i in range (0, len(mmds2), 1): # берем тестовое фото len(mmds2)
        iTest += 1
        minDist = 100
        for j in range (0, len(mmds1),): # ищем его среди эталонных 
            vector1 = mmds1[j] # splitMMDS()[2][i]
            vector2 = mmds2[i] # splitMMDS()[0][j]
            evk = evklidDistanse (vector1, vector2)
            if (evk < minDist):
                minDist = evk
                jv = j
                evk = 0            
        if (testDS[i] == trainDS[jv]):
            acc += 1
    return (acc / iTest)

In [294]:
print ("accuracy = ", minDist())

accuracy =  0.95
